In [3]:
# recreation from https://github.com/pytorch/examples/blob/main/mnist/main.py
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x) # 36864
        x = F.relu(x)
        #x = F.max_pool2d(x, 2)
        #x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [11]:
f = nn.Conv2d(1, 32, 3, 1)
f2 = nn.Conv2d(32, 64, 3, 1)
l = nn.Linear(64*396*396, 1)
f2(f(torch.ones(1,103,103))).shape
l(nn.Flatten(f2(f(torch.ones(1,400,400))),1))


TypeError: linear(): argument 'input' (position 1) must be Tensor, not Flatten

In [71]:
def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
def count_parameters(model):
    # https://discuss.pytorch.org/t/how-do-i-check-the-number-of-parameters-of-a-model/4325/7
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [72]:
# Training settings
batch_size = 64
use_mps = True and torch.backends.mps.is_available()
test_batch_size = 1000
epochs = 14
lr = 1.0
gamma = 0.7
seed = 1
log_interval = 10
save_model = False



In [73]:
if use_mps:
    device = torch.device("mps")
else:
    device = torch.device("cpu")
    
train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}

transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [74]:
for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, epoch, log_interval)
        test(model, device, test_loader)
        scheduler.step()

if save_model:
    torch.save(model.state_dict(), "mnist_cnn.pt")

RuntimeError: linear(): input and weight.T shapes cannot be multiplied (64x36864 and 9216x128)

In [69]:
count_parameters(model)

1199882

# Results
Default PyTorch MNIST example:
```
Test set: Average loss: 0.0226, Accuracy: 9929/10000 (99,3%)
Parameter count: 1 199 882
```
After removing dropout and maxpool layers (simplification to CNN and dense layers) we see a slight decrease in performance
```
Test set: Average loss: 0.0459, Accuracy: 9899/10000 (99,0%)
Parameter count: 4 738 826
```
After adjusting the first CNN to larger output channels (53) we see a slight increase in performance
```
Test set: Average loss: 0.0478, Accuracy: 9902/10000 (99,0%)
Parameter count: 4 751 132
```

# Rules
- CNN input of layer = output of previous
  ```
  self.conv1 = nn.Conv2d(1, 32, 3, 1)
  self.conv2 = nn.Conv2d(32, 64, 3, 1)
  ```
- After CNN, Flatten size depends on input size
- Adding maxpooling layers reduces parameter count

# Idea
- Get shape of the input
- When building up the dna sequence, it needs to be passed through

# DNA codes:
- C - Conv layer
- F - Flatten layer, automatically injected between Conv and Dense layer - Does not appear in DNA code
- D - Dense layer
- P - Maxpool layer
- R - Dropout layer

# Definitions:
- C<in_channels>,<out_channels><kernel_size><stride> - in_channels = 1 if first layer or Dense, otherwwise, it matches the previous layer's out_channels
- D<insize><outsize> - insize = flattened input size OR output of previous layer
- P<kernel_size>

# DNA of the default PyTorch MNIST example
input: Tensor(1x28x28)
```
C,1,32,3,1
C,32,64,3,1
P,2
D,9216,128
D,128,10
```
output: Tensor(1x10) -> maxpooling is not included in the output as it depends on the task, not evolution